## Improving ROCrate Upload Preformance with Generators 

In [5]:
import os
import sys
import pathlib

testPaths = pathlib.Path(".").absolute()
sys.path.append(str(testPaths.parent.parent))

from fairscape_mds.core.config import appConfig

## Test Setup

In [ ]:
# given a user, and an rocrate to upload


## Implementation

In [6]:
from fairscape_mds.models.user import UserWriteModel
from fairscape_models.model_card import ModelCard
import httpx

In [9]:
root_url = "http://localhost:8080/api"

testUser = UserWriteModel.model_validate({
    "email": "test@example.org",
    "firstName": "John",
    "lastName": "Doe",
    "password": "test"
    })

# upload ml model just metadata
modelInstance = ModelCard.model_validate({
    "@id": "ark:59853/test-model",
    "@type": "EVI:MLModel",
    "name": "example models",
    "description": "a fake ml model card for testing",
    "author": "Max Levinson",
    "keywords": ["test", "example"],
    "version": "0.1.0",
    "modelType": "Image Classification/Feature Backbone",
    "framework": "Pytorch",
    "modelFormat": "safetensor",
    "generatedBy": {"@id": "ark:59853/training-computation"},
    "trainingDataset": "https://huggingface.co/datasets/ILSVRC/imagenet-1k",
    "parameters": "8000000",
    "inputSize": "224x224",
    "indendedUseCase": None,
    "usageInformation": None,
    "contentUrl": "https://huggingface.co/timm/densenet121.tv_in1k",
    "url": "https://huggingface.co/timm/densenet121.tv_in1k",
    "dataLicense": None,
    "citation": "Densely Connected Convolutional Networks: https://arxiv.org/abs/1608.06993"
})

In [11]:
# login user
loginData = {
    "username": testUser.email,
    "password": testUser.password
    }


loginResponse = httpx.post(
    root_url + "/login", 
    data=loginData
    )
loginJSON = loginResponse.json()

assert loginResponse.status_code == 200
assert loginJSON 
assert loginJSON.get("access_token")


authHeaders = {
    "Authorization": f"Bearer {loginJSON.get('access_token')}"
    }

In [14]:
root_url + "/mlmodel"

'http://localhost:8080/api/mlmodel'

In [ ]:
# post ml model

inputForm = {
	"metadata": modelInstance.model_dump_json(by_alias=True),
  #"content": None
}

# upload a test rocrate
uploadResponse = httpx.post(
      root_url + "/mlmodel",
      data= inputForm,
      headers=authHeaders,
      timeout=30
)

uploadJSON = uploadResponse.json()

In [ ]:
uploadJSON

{'@id': 'ark:59853/test-model',
 '@type': 'https://w3id.org/EVI#MLModel',
 'name': 'example models'}

In [48]:

files = [('content', open('crud/data/example.csv', 'rb'))]

modelInstanceContent = ModelCard.model_validate({
    "@id": "ark:59853/test-model-2",
    "@type": "EVI:MLModel",
    "name": "example models",
    "description": "a fake ml model card for testing",
    "author": "Max Levinson",
    "keywords": ["test", "example"],
    "version": "0.1.0",
    "modelType": "Image Classification/Feature Backbone",
    "framework": "Pytorch",
    "modelFormat": "safetensor",
    "generatedBy": {"@id": "ark:59853/training-computation"},
    "trainingDataset": "https://huggingface.co/datasets/ILSVRC/imagenet-1k",
    "parameters": "8000000",
    "inputSize": "224x224",
    "indendedUseCase": None,
    "usageInformation": None,
    "contentUrl": "https://huggingface.co/timm/densenet121.tv_in1k",
    "url": "https://huggingface.co/timm/densenet121.tv_in1k",
    "dataLicense": None,
    "citation": "Densely Connected Convolutional Networks: https://arxiv.org/abs/1608.06993"
})

inputForm = {
	"metadata": modelInstanceContent.model_dump_json(by_alias=True),
}

# upload a test rocrate
uploadResponse = httpx.post(
      root_url + "/mlmodel",
      data= inputForm,
      files=files,
      headers=authHeaders,
      timeout=30
)

uploadJSON = uploadResponse.json()

In [49]:
uploadJSON

{'@id': 'ark:59853/test-model-2',
 '@type': 'https://w3id.org/EVI#MLModel',
 'name': 'example models'}